In [1]:
# Required for importing modules from parent directory
import os
import sys

current_dir = os.path.dirname(os.path.abspath("__file__"))
parent_dir = os.path.dirname(current_dir)
print(parent_dir)
sys.path.append(parent_dir)

c:\Users\I551674\Desktop\6. Semester\Projektrealisierung


In [2]:
from pathlib import Path
import pandas as pd

from src.loader import TripLoader

pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

In [3]:
trip_loader = TripLoader(fix_csv_errors=False)
trips_data = [trip_loader.trips_ABCD, trip_loader.trips_MNOP, trip_loader.trips_ZYXW]
combined_trips = pd.concat(trips_data, ignore_index=True)

In [ ]:
combined_trips = combined_trips.drop(columns=["entry_details", "header_line"])
combined_trips.head()

In [7]:
combined_trips[(combined_trips["flight_number"] == 2127) & (combined_trips["flight_date"] == 2)].head(5)

id        creation_time airline_code  flight_number  flight_date  \
4     33902706  2024-05-01 09:14:41           AB           2127            2   
285   33846055  2024-04-30 22:14:14           AB           2127            2   
3874  33990350  2024-05-02 08:01:50           AB           2127            2   
4770  33946948  2024-05-01 23:36:13           AB           2127            2   
5658  33999332  2024-05-02 09:45:23           AB           2127            2   

     departure_airport     user_name      action_name  \
4                  BOM  service-acco  ASMMsgProcessor   
285                BOM  service-acco  ASMMsgProcessor   
3874               BOM  service-acco  ASMMsgProcessor   
4770               BOM  service-acco  ASMMsgProcessor   
5658               BOM  service-acco  ASMMsgProcessor   

                                                                                                                                                                                                             header_line  \
4     2024-05-01 09:14:41,884 INFO  [829566f527b588fa] [AB2127 /02 BOM | service-acco | ASMMsgProcessor                     |                                | Thread-49765 (ActiveMQ-client-global-threads)]  Received:   
285   2024-04-30 22:14:14,006 INFO  [15f006af1588e82b] [AB2127 /02 BOM | service-acco | ASMMsgProcessor                     |                                | Thread-49452 (ActiveMQ-client-global-threads)]  Received:   
3874  2024-05-02 08:01:50,890 INFO  [fd962196909664d8] [AB2127 /02 BOM | service-acco | ASMMsgProcessor                     |                                | Thread-50320 (ActiveMQ-client-global-threads)]  Received:   
4770  2024-05-01 23:36:13,364 INFO  [3cb4f40e7728d87b] [AB2127 /02 BOM | service-acco | ASMMsgProcessor                     |                                | Thread-50161 (ActiveMQ-client-global-threads)]  Received:   
5658  2024-05-02 09:45:23,225 INFO  [cfe5c7aff039f5d0] [AB2127 /02 BOM | service-acco | ASMMsgProcessor                     |                                | Thread-50384 (ActiveMQ-client-global-threads)]  Received:   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [15]:
combined_trips["departure_airport"].unique()

array(['BLR', 'BOM', 'DEL', 'GOX', 'IXB', 'COK', 'CCU', 'HYD', 'GAU',
       'SXR', 'AMD', 'VNS', 'PNQ', 'LKO', 'AYJ', 'IXZ', 'GWL', 'BBI',
       'DOH', 'IXA', 'MAA', 'SIN', 'DXB', 'SIG', 'RUP', 'DUB', 'LAX',
       'ORK', 'LHR', 'IAD', 'MAD', 'MUC', 'MAN', 'AGP', 'YYZ', 'ORD',
       'JFK', 'NOC', 'SFO', 'SNN', 'EWR', 'MCO', 'BDL', 'BOS', 'PHL',
       'SEA', 'CLE', 'VCE', 'CDG', 'FAO', 'BGI', 'MSP', 'PGF', 'NAP',
       'BOD', 'FCO', 'DUS', 'TLS', 'CMH', 'LPA', 'BVB', 'SSA', 'REC',
       'MCZ', 'SLZ', 'NAT', 'STM', 'CGB', 'IMP', 'PVH', 'MAO', 'CGR',
       'GRU', 'VIX', 'THE', 'MOC', 'SJP', 'FOR', 'GYN', 'UDI', 'CWB',
       'CNF', 'POA', 'BSB', 'BEL', 'GIG', 'PPB', 'FLN', 'IGU', 'AJU',
       'CGH', 'NVT', 'VCP', 'JDO', 'SDU', 'IPN', 'ORY', 'RAO', 'LIS',
       'FEN', 'FLL', 'LDB', 'UBA', 'CAC', 'PNZ', 'JPA', 'BPS', 'IZA',
       'ARU', 'VDC', 'JOI', 'JTC', 'MGF', 'PFB', 'CMG', 'CAW', 'URG',
       'BYO', 'CKS', 'BVH', 'LEC', 'OPS', 'MAB', 'SJL', 'MII', 'OAL',
       'MCP', 'IOS',

In [14]:
airport_flight_counts = combined_trips.groupby('departure_airport')['flight_number'].nunique().reset_index()
airport_flight_counts.columns = ['departure_airport', 'unique_flight_numbers']

In [16]:
import folium
from src.utils import airport_coords
# Initialize a map centered around a geographical location (Brazil in this case)
map_center = [-14.2350, -51.9253]  # Center of Brazil
m = folium.Map(location=map_center, zoom_start=4)

# Add markers to the map
for _, row in airport_flight_counts.iterrows():
    airport = row['departure_airport']
    count = row['unique_flight_numbers']
    coords = airport_coords.get(airport)
    if coords:
        folium.Marker(
            location=coords,
            popup=f'{airport}: {count} unique flight numbers',
            tooltip=f'{airport}: {count} unique flight numbers'
        ).add_to(m)

# Save the map to an HTML file
m.save('airport_flight_map.html')